In [ ]:
from rembg import remove, new_session
from pathlib import Path
import pandas as pd
import numpy as np
from PIL import ImageEnhance, Image
import cv2

# def remove_background_and_mask(image_path, theshold=20):
#     image = Image.open(image_path).convert("RGB")
#     r = remove(image, session=session)  # Gibt RGBA-Bild zurück
#     rgba = r.convert("RGBA")

#     # Extrahiere Alpha-Kanal
#     alpha = rgba.getchannel("A")  # Graubild: 0 (transparent) – 255 (sichtbar)

#     # Erzeuge Binärmaske aus Alpha: alles > 0 = 255 (Objekt), sonst 0
#     mask = alpha.point(lambda x: 255 if x > theshold else 0).convert("L")

#     # Ersetze transparenten Hintergrund mit Schwarz (für Trainingsbild)
#     background = Image.new("RGBA", rgba.size, (0, 0, 0, 255))
#     composited = Image.alpha_composite(background, rgba)

#     # In Grayscale konvertieren
#     grayscale = composited.convert("L")

#     return grayscale, mask


def remove_background_and_mask(image_path, threshold=50):
    # Lade Originalbild
    image_original = Image.open(image_path).convert("RGB")

    # Erzeuge Kopie zur Kontrastverbesserung nur für Segmentierung
    image_preproc = image_original.copy()
    
    # Kontrast erhöhen
    # enhancer = ImageEnhance.Contrast(image_preproc)
    # image_preproc = enhancer.enhance(2.0)

    # # Schärfe erhöhen
    # sharpener = ImageEnhance.Sharpness(image_preproc)
    # image_preproc = sharpener.enhance(1.5)

    # Segmentiere vorverarbeitetes Bild
    rgba = remove(image_preproc, session=session).convert("RGBA")

    # Extrahiere Alpha-Kanal und baue Binärmaske
    alpha = rgba.getchannel("A")

    # Maske in NumPy (binär)
    mask_np = np.array(alpha).astype(np.uint8)
    binary_mask = (mask_np > 0).astype(np.uint8)  # 1 = Maus, 0 = Hintergrund

    # Wende Maske auf das ORIGINAL-Bild an
    orig_np = np.array(image_original)
    orig_np[~binary_mask.astype(bool)] = 0  # Hintergrund auf schwarz setzen

    # Konvertiere zu PIL und Graustufen
    grayscale_img = Image.fromarray(orig_np).convert("L")
    mask_img = Image.fromarray(binary_mask * 255)

    return grayscale_img, mask_img

In [ ]:


DIR = Path().resolve()
PROJECT_ROOT = Path().resolve().parent.parent

LABEL_CSV = PROJECT_ROOT / "data" / "MGS_data" / "labels" / "labels.csv"
IMAGE_DIR = PROJECT_ROOT / "data" / "MGS_data" / "data"
OUTPATH = DIR / "outputs_rbg_u2net3"

session = new_session("isnet-general-use")

df = pd.read_csv(LABEL_CSV)
for file, label in zip(df["filename"], df["label"]):
    image_path = IMAGE_DIR / file
    if not image_path.exists():
        print(f"Image {image_path} does not exist, skipping.")
        continue

    print(f"Processing {image_path} with label {label}")
    r_image, mask = remove_background_and_mask(image_path, threshold=10)

    out_image_path = OUTPATH / "images" / file
    out_mask_path = OUTPATH / "masks" / file

    out_image_path.parent.mkdir(parents=True, exist_ok=True)
    out_mask_path.parent.mkdir(parents=True, exist_ok=True)

    r_image.save(out_image_path)
    mask.save(out_mask_path)

    print(f"✅ Saved processed image to {out_image_path}")
    print(f"✅ Saved mask to {out_mask_path}")
    


In [ ]:
folder = r"C:\Users\Freun\Desktop\htcv_mgs\notebooks\improvements\best_rbg"
for fname in os.listdir(folder):
    if " - Kopie" in fname:
        new_name = fname.replace(" - Kopie", "")
        src = os.path.join(folder, fname)
        dst = os.path.join(folder, new_name)
        os.rename(src, dst)
        print(f"✅ Renamed: {fname} → {new_name}")

In [37]:
import os
import shutil
from pathlib import Path

image_nb_dir = Path(r"C:\Users\Freun\Desktop\htcv_mgs\notebooks\improvements\best_rbg")
image_image_dir = Path(r"C:\Users\Freun\Desktop\htcv_mgs\notebooks\improvements\best_rbg\images")
mask_image_dir = Path(r"C:\Users\Freun\Desktop\htcv_mgs\notebooks\improvements\best_rbg\masks")

img_dir = Path(r"C:\Users\Freun\Desktop\htcv_mgs\notebooks\improvements\outputs_rbg_u2net2\images")
mask_dir = Path(r"C:\Users\Freun\Desktop\htcv_mgs\notebooks\improvements\best_rbg\missing_masks")

os.mkdir(image_nb_dir) if not os.path.exists(image_nb_dir) else None
os.mkdir(image_image_dir) if not os.path.exists(image_image_dir) else None
os.mkdir(mask_image_dir) if not os.path.exists(mask_image_dir) else None

for image_name in os.listdir(image_nb_dir):
    if image_name.lower().endswith((".jpg", ".jpeg", ".png")):
        src_image = image_nb_dir / image_name
        src_mask = mask_dir / image_name
        dst_image = image_image_dir / image_name
        dst_mask = mask_image_dir / image_name

        # Bild kopieren
        shutil.copy(src_image, dst_image)
        shutil.copy(src_mask, dst_mask) 

# I have all images in a directory that looked good. For the rest I would like to increase the threshold of the alpha channel to 50, to increase the visibility of the mice.



In [38]:
import pandas as pd
from PIL import Image
from pathlib import Path

# Daten & Pfade
LABEL_CSV = Path(PROJECT_ROOT) / "data" / "MGS_data" / "labels" / "labels.csv"
IMAGE_DIR = Path(PROJECT_ROOT) / "data" / "MGS_data" / "data"
OUTPATH = Path(image_nb_dir)

# 1. Alle bereits geprüften Bildnamen sammeln
approved_names = set(os.listdir(image_image_dir))

# 2. CSV einlesen und gefilterte Labels extrahieren
df = pd.read_csv(LABEL_CSV)
remaining = df[~df["filename"].isin(approved_names)]

# 3. Re-Processing mit Alpha-Threshold > 0 (z. B. 50)
for file, label in zip(remaining["filename"], remaining["label"]):
    # image_path = IMAGE_DIR / file
    # if not image_path.exists():
    #     print(f"❌ Image {image_path} missing. Skipping.")
    #     continue

    # print(f"🛠️ Processing (retry) {image_path} with alpha threshold 50")
    # try:
    #     r_image, mask = remove_background_and_mask(image_path, threshold=0)
    # except Exception as e:
    #     print(f"⚠️ Failed to process {file}: {e}")
    #     continue

    # out_image_path = OUTPATH / "missing_images" / file
    # out_mask_path = OUTPATH  / "missing_masks" / file

    # out_image_path.parent.mkdir(parents=True, exist_ok=True)
    # out_mask_path.parent.mkdir(parents=True, exist_ok=True)

    # r_image.save(out_image_path)
    # mask.save(out_mask_path)

    # print(f"✅ Saved reprocessed image to {out_image_path}")
    
    image_path = IMAGE_DIR / file
    if not image_path.exists():
        print(f"❌ Image {image_path} missing. Skipping.")
        continue

    print(f"🛠️ Copying original as fallback for {file}")
    try:
        # Lade Originalbild
        original = Image.open(image_path).convert("RGB")
        grayscale = original.convert("L")

        # Erzeuge Dummy-Maske (weiß = alles sichtbar)
        width, height = grayscale.size
        mask_np = np.ones((height, width), dtype=np.uint8) * 255
        mask = Image.fromarray(mask_np)

    except Exception as e:
        print(f"⚠️ Failed to process {file}: {e}")
        continue

    out_image_path = OUTPATH / "missing_images" / file
    out_mask_path = OUTPATH / "missing_masks" / file

    out_image_path.parent.mkdir(parents=True, exist_ok=True)
    out_mask_path.parent.mkdir(parents=True, exist_ok=True)

    grayscale.save(out_image_path)
    mask.save(out_mask_path)

    print(f"✅ Saved fallback image to {out_image_path}")

🛠️ Copying original as fallback for lw_1-17a.JPG
✅ Saved fallback image to C:\Users\Freun\Desktop\htcv_mgs\notebooks\improvements\best_rbg\missing_images\lw_1-17a.JPG
🛠️ Copying original as fallback for lw_1-4b.JPG
✅ Saved fallback image to C:\Users\Freun\Desktop\htcv_mgs\notebooks\improvements\best_rbg\missing_images\lw_1-4b.JPG
🛠️ Copying original as fallback for lw_11-10b.JPG
✅ Saved fallback image to C:\Users\Freun\Desktop\htcv_mgs\notebooks\improvements\best_rbg\missing_images\lw_11-10b.JPG
🛠️ Copying original as fallback for lw_11-11c.JPG
✅ Saved fallback image to C:\Users\Freun\Desktop\htcv_mgs\notebooks\improvements\best_rbg\missing_images\lw_11-11c.JPG
🛠️ Copying original as fallback for lw_11-12a.JPG
✅ Saved fallback image to C:\Users\Freun\Desktop\htcv_mgs\notebooks\improvements\best_rbg\missing_images\lw_11-12a.JPG
🛠️ Copying original as fallback for lw_11-13a.JPG
✅ Saved fallback image to C:\Users\Freun\Desktop\htcv_mgs\notebooks\improvements\best_rbg\missing_images\lw_11-